In [5]:
with open('data/coincidences.tex', 'r') as f:
    txt = f.read()

In [39]:
import json, requests
import numpy as np
import pandas as pd
from collections import OrderedDict

In [28]:
rows = [tuple(r.strip("\\\\").split(" &")) for r in txt.split('\n')]
tab = pd.DataFrame(np.array(
    rows, 
    dtype = [
        ('name', 'U30'), 
        ('discovery', '<f8'),
        ('nu_alert', 'U30'), 
        ('alert_epoch', '<f8'),
        ('dt', '<f8'),
        ('classf', 'U30')
    ]
))

In [63]:
sn1a_m = tab.classf == 'SN~Ia'
time_m = (tab.dt > -100) & (tab.dt < 200)

small_tab = tab[sn1a_m & time_m]

In [64]:
small_tab

,name,discovery,nu_alert,alert_epoch,dt,classf,tns_class
3,SN~2021vpv,2459439.0,IC210608A,2459373.7,65.0,SN~Ia,SN Ia-91T-like
25,SN~2020aaxo,2459187.8,IC200523A,2458992.6,195.0,SN~Ia,
27,SN~2020yxd,2459170.8,IC200614A,2459015.0,156.0,SN~Ia,
29,SN~2020unl,2459138.1,IC200425A,2458965.5,173.0,SN~Ia,
34,SN~2020oye,2459054.8,IC200410A,2458950.5,104.0,SN~Ia,
36,SN~2020lsc,2459011.9,IC200410A,2458950.5,61.0,SN~Ia,SN Ia
37,SN~2020kpx,2458994.8,IC191119A,2458806.5,188.0,SN~Ia,
38,SN~2020joh,2458985.8,IC200410A,2458950.5,35.0,SN~Ia,SN Ia
44,AT2020fhs,2458943.1,IC200410A,2458950.5,-7.0,SN~Ia,SN Ia
45,SN~2020cli,2458909.9,IC200410A,2458950.5,-41.0,SN~Ia,SN Ia


In [40]:
TNS                 = "www.wis-tns.org"
#TNS                 = "sandbox.wis-tns.org"
url_tns_api         = "https://" + TNS + "/api/get"

TNS_BOT_ID          = "115364"
TNS_BOT_NAME        = "ZTF_DESY"
TNS_API_KEY         = "d1971f402dc6d10eaa214708db1d5db2bbd06aaa"

tns_marker = 'tns_marker{"tns_id":115364,"type": "bot", "name":"ZTF_DESY"}'

def get(getObj):
    get_url = url_tns_api + "/object"
    headers = {'User-Agent': tns_marker}
    json_file = OrderedDict(getObj)
    get_data = {'api_key': TNS_API_KEY, 'data': json.dumps(json_file)}
    response = requests.post(get_url, headers = headers, data = get_data)
    return response

def get_tns_info(at_name):
    name = at_name.replace(' ', '').split("AT")[1]
    get_obj = [("objname", name), ("objid", ""), ("photometry", "0"), ("spectra", "0")]
    response = get(get_obj)
    print(f"{response.url} {response.status_code}: {response.reason}")
    d = json.loads(response.text)
    return d

In [45]:
tab["tns_class"] = ""

In [65]:
tnsd = {}
for i, r in small_tab.iterrows():
    n = r['name']
    print(n)
    if "SN~" in n:
        n = "AT" +  n.split("SN~")[-1]
        print("  actually " + n)
    
    tns_info = get_tns_info(n)
    tnsd[n] = tns_info
    
    small_tab.loc[i, "tns_class"] = tns_info['data']['reply']['object_type']['name']

SN~2021vpv
  actually AT2021vpv
https://www.wis-tns.org/api/get/object 200: OK
SN~2020aaxo
  actually AT2020aaxo


/afs/ifh.de/user/n/neckerja/scratch/envs/nu_stat/lib/python3.9/site-packages/pandas-1.3.2-py3.9-linux-x86_64.egg/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


https://www.wis-tns.org/api/get/object 200: OK
SN~2020yxd
  actually AT2020yxd
https://www.wis-tns.org/api/get/object 200: OK
SN~2020unl
  actually AT2020unl
https://www.wis-tns.org/api/get/object 200: OK
SN~2020oye
  actually AT2020oye
https://www.wis-tns.org/api/get/object 200: OK
SN~2020lsc
  actually AT2020lsc
https://www.wis-tns.org/api/get/object 200: OK
SN~2020kpx
  actually AT2020kpx
https://www.wis-tns.org/api/get/object 200: OK
SN~2020joh
  actually AT2020joh
https://www.wis-tns.org/api/get/object 200: OK
AT2020fhs
https://www.wis-tns.org/api/get/object 200: OK
SN~2020cli
  actually AT2020cli
https://www.wis-tns.org/api/get/object 200: OK
SN~2020zj
  actually AT2020zj
https://www.wis-tns.org/api/get/object 200: OK
SN~2019zhs
  actually AT2019zhs
https://www.wis-tns.org/api/get/object 200: OK
SN~2019stx
  actually AT2019stx
https://www.wis-tns.org/api/get/object 200: OK
ASASSN-19kw


IndexError: list index out of range

In [66]:
small_tab

,name,discovery,nu_alert,alert_epoch,dt,classf,tns_class
3,SN~2021vpv,2459439.0,IC210608A,2459373.7,65.0,SN~Ia,SN Ia-91T-like
25,SN~2020aaxo,2459187.8,IC200523A,2458992.6,195.0,SN~Ia,SN Ia
27,SN~2020yxd,2459170.8,IC200614A,2459015.0,156.0,SN~Ia,SN Ia
29,SN~2020unl,2459138.1,IC200425A,2458965.5,173.0,SN~Ia,SN Ia
34,SN~2020oye,2459054.8,IC200410A,2458950.5,104.0,SN~Ia,SN Ia
36,SN~2020lsc,2459011.9,IC200410A,2458950.5,61.0,SN~Ia,SN Ia
37,SN~2020kpx,2458994.8,IC191119A,2458806.5,188.0,SN~Ia,SN Ia
38,SN~2020joh,2458985.8,IC200410A,2458950.5,35.0,SN~Ia,SN Ia
44,AT2020fhs,2458943.1,IC200410A,2458950.5,-7.0,SN~Ia,SN Ia
45,SN~2020cli,2458909.9,IC200410A,2458950.5,-41.0,SN~Ia,SN Ia


In [77]:
print(small_tab.to_string(index=False, columns=['name', 'dt', 'tns_class'], justify='right'))

       name    dt      tns_class
 SN~2021vpv  65.0 SN Ia-91T-like
SN~2020aaxo 195.0          SN Ia
 SN~2020yxd 156.0          SN Ia
 SN~2020unl 173.0          SN Ia
 SN~2020oye 104.0          SN Ia
 SN~2020lsc  61.0          SN Ia
 SN~2020kpx 188.0          SN Ia
 SN~2020joh  35.0          SN Ia
  AT2020fhs  -7.0          SN Ia
 SN~2020cli -41.0          SN Ia
  SN~2020zj -74.0          SN Ia
 SN~2019zhs  45.0      SN Ia-pec
 SN~2019stx -24.0          SN Ia
ASASSN-19kw 113.0               
 SN~2019crd -92.0          SN Ia
